# L1d: Fun With Base 16 Numbers
As a scientist or engineer, it's strange that (arguably) the most critical numbering systems for data science and machine learning applications are not floating point numbers but base $b$ integers! What? Yep. It's true.

Base b integers are used in many hard-to-see, yet critical, applications. Let's dig into one of these in the activity, namely, the mathematical encoding of text.
* __Backstory__. Computers don't see text like us, i.e., as a collection of Strings, happy face emojis, and other punctuation characters. Instead, computers only see sequences of binary bits. Thus, they must map each character in text to a unique integer, i.e., a sequence of binary bits. Because it's hard for us to think in binary, we express that integer in some base-b positional numbering system, resulting in text being just another digit sequence. 
* __Computers only see binary numbers, gotcha, but why base b__? Aren't base 10 numbers enough? Yes, they are, but remember that the Unicode code points we saw earlier can be up to 4 bytes (32 bits). It's hard for humans to deal with long sequences of 0s and 1s. Thus, encoding numbers in base-b, e.g., base 16, facilitates efficient text encoding by representing characters as fixed-width numerical representations easily decoded back into readable symbols.

Let's start at the beginning. 
* Traditionally, characters were represented using [the ASCII system](https://en.wikipedia.org/wiki/ASCII). [Standard (7-bit) ASCII](https://www.ascii-code.com/ASCII) uses 7 bits of storage and defines 128 character codes numbered 0 through 127 (each character is mapped to a number ranging between 0 and 127). [Extended (8-bit) ASCII variants](https://www.ascii-code.com) is a similar idea, but uses 8 bits and defines 256 values numbered 0 to 255. But is 255 characters enough?
* No! Modern encodings, such as [Unicode encodings like UTF-8 or UTF-16](https://en.wikipedia.org/wiki/Unicode), represent a much wider range of characters and symbols.

In this lab, we'll dig into the numerical basis of text, starting with the [7-bit ASCII system](https://en.wikipedia.org/wiki/ASCII) and working our way to [Unicode encodings like UTF-8 or UTF-16](https://en.wikipedia.org/wiki/Unicode). 

___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include__: The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/).

For clarity, `include(joinpath(@__DIR__, "Include.jl"))` loads the `Include.jl` file that lives next to this notebook. `@__DIR__` expands to the directory containing the notebook, so this call works even if you open the notebook from a different working directory in VSCode.

In [2]:
include(joinpath(@__DIR__, "Include.jl")); # what is going on here?

    Updating git-repo `https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl.git`
   Resolving package versions...
   Installed x265_jll ───────────────────── v4.1.0+0
   Installed libfdk_aac_jll ─────────────── v2.0.4+0
   Installed Libmount_jll ───────────────── v2.41.1+0
   Installed JpegTurbo_jll ──────────────── v3.1.2+0
   Installed GR_jll ─────────────────────── v0.73.17+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed OffsetArrays ───────────────── v1.17.0
   Installed LERC_jll ───────────────────── v4.0.1+0
   Installed Opus_jll ───────────────────── v1.5.2+0
   Installed RelocatableFolders ─────────── v1.0.1
   Installed LoggingExtras ──────────────── v1.1.0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.7+0
   Installed Measures ───────────────────── v0.3.2
   Installed FFTW ───────────────────────── v1.9.0
   Installed Unitful ────────────────────── v1.24.0
   Installed ConcurrentUtilities ────────── v2.5.0
   Installed Xorg_xcb_util_wm_jl

___

##  Task 1: Explore the ASCII Character Set
Let's start by considering the representation of the characters in the 7-bit ASCII character system. 

> __Fun fact__: While 7-bit ASCII has been replaced a long time ago, it still lives on in the modern [Unicode system](https://en.wikipedia.org/wiki/Unicode). The original ASCII characters are encoded as the first 128 characters (`0`$\rightarrow$`127`) in [the Unicode system](https://en.wikipedia.org/wiki/Unicode).

Let's start by building the `ascii_char_dictionary::Dict{Int64, Char}` dictionary, which is a mapping between the ASCII character index (an integer) and the character value, which [is type `c::Char` in Julia](https://docs.julialang.org/en/v1/base/strings/#Core.Char).

This logic involves some advanced tools and techniques we have yet to discuss. You can skip the implementation details for now; we'll come back to them later. However, there is one interesting method, namely [the `convert(...)` method](https://docs.julialang.org/en/v1/base/base/#Base.convert). This is a direct way to convert between the characters that you and I see and the integers that the computer understands.

In [3]:
ascii_char_dictionary = let

    # initialize -
    ascii_char_dictionary = Dict{Int64, Char}(); # storage for index - character map
    ASCII_character_range = range(0,stop=127,step=1) |> collect; # 7-bit ASCII indexes

    # main loop -
    for i ∈ eachindex(ASCII_character_range)
        my_ascii_char_index = ASCII_character_range[i];
        c = convert(Char, my_ascii_char_index) # hmmm. This is an interesting function.
        ascii_char_dictionary[my_ascii_char_index] = c;
    end
    ascii_char_dictionary;
end

Dict{Int64, Char} with 128 entries:
  5   => '\x05'
  56  => '8'
  35  => '#'
  55  => '7'
  110 => 'n'
  114 => 'r'
  123 => '{'
  60  => '<'
  30  => '\x1e'
  32  => ' '
  6   => '\x06'
  67  => 'C'
  45  => '-'
  117 => 'u'
  73  => 'I'
  115 => 's'
  112 => 'p'
  64  => '@'
  90  => 'Z'
  ⋮   => ⋮

`Unhide` the code block to see how we build a table of the ASCII characters using [the `pretty_table(...)` function exported by the PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl). Let's look at what the `ASCII` characters are. 
* This logic involves some advanced tools and techniques we have yet to discuss. You can skip the implementation details for now; we'll come back to them later. TLDR: We build the rows in the table using a [`for-loop`](https://docs.julialang.org/en/v1/base/base/#for) and [the `convert(...)` function](https://docs.julialang.org/en/v1/base/base/#Base.convert), where we push the data for a row into a [DataFrame](https://dataframes.juliadata.org/stable/), and then display the data by [calling the `pretty_table(...)` method](https://github.com/ronisbr/PrettyTables.jl).

In [3]:
let
    ASCII_index_array = keys(ascii_char_dictionary) |> collect |> sort;
    character_table_df = DataFrame();
    for i ∈ eachindex(ASCII_index_array)
        my_ascii_char_index = ASCII_index_array[i];
        c = ascii_char_dictionary[my_ascii_char_index];

        row = (
            i = my_ascii_char_index,
            character = c
        ); # -> what is going on here? This is a cool type called a NamedTuple ...
        push!(character_table_df,row); # push! -> what is going on here?
    end
    pretty_table(character_table_df, tf=tf_simple)
end

======== ============
      i   character 
  Int64        Char 
======== ============
      0          \0
      1        \x01
      2        \x02
      3        \x03
      4        \x04
      5        \x05
      6        \x06
      7          \a
      8          \b
      9          \t
     10          \n
     11          \v
     12          \f
     13          \r
     14        \x0e
     15        \x0f
     16        \x10
     17        \x11
     18        \x12
     19        \x13
     20        \x14
     21        \x15
     22        \x16
     23        \x17
     24        \x18
     25        \x19
     26        \x1a
     27          \e
     28        \x1c
     29        \x1d
     30        \x1e
     31        \x1f
     32
     33           !
     34           "
     35           #
     36           $
     37           %
     38           &
     39           '
     40           (
     41           )
     42           *
     43           +
     44           ,
     45           -
     4

In the code block above, we explicitly called [the `convert(...)` method](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/) to convert an [`Int`](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Integers) to a [`Char` type](https://docs.julialang.org/en/v1/base/strings/#Core.Char). However, this was unnecessary, as Julia (and most modern languages) will automatically try to understand what you type (and do conversions for you if it can). For example:

In [4]:
'c' |> Int # wow. This seems a little magical. What is going on here?? (notice the single quotes), pick a different char?

99

In [5]:
Int('c') # Unpack the |> We can convert between ASCII characters and Int! 

99

In [6]:
Char(99)

'c': ASCII/Unicode U+0063 (category Ll: Letter, lowercase)

### Limitations of ASCII
Wow! ASCII seems pretty cool. It only uses 7 bits (which is super simple) to encode 128 characters (numbers, small and capital letters, some math symbols, etc). Why would we ever need anything beyond these 127 characters? As it turns out, there are many reasons. Let's look at three:
* The original 7-bit ASCII was limited to 128 characters (0–127), so it couldn’t represent characters beyond basic English letters, digits, punctuation, and control codes.
* The original 7-bit ASCII character set offered no native support for accented or non-Latin characters (e.g., Cyrillic, Greek, and Arabic), hindering internationalization. It only represented English.
* The original 7-bit ASCII character set omitted modern typographic symbols, mathematical glyphs, and emojis, making it inadequate for rich text or graphical communication. For example, it can't be used to represent rich mathematical text. Bummer!

These limitations, and others, led to the development of [the Unicode system](https://en.wikipedia.org/wiki/Unicode).

## Task 2: Explore the Unicode Character Set
Unlike the original 7-bit ASCII character set, which has only 128 characters, the modern [Unicode system](https://en.wikipedia.org/wiki/Unicode) _can encode_ 1,114,112 possible characters (code points). However, as of [Unicode Standard version 16.0 (released September 10, 2024)](https://www.unicode.org/versions/Unicode16.0.0/), only 154,998 of those possible code points are assigned to unique characters; various characters in many languages, a much larger family of mathematical symbols, and [emoji](https://en.wikipedia.org/wiki/Emoji) of all sorts are currently assigned. There is plenty of room to grow!

Technical:
* The [Unicode system](https://en.wikipedia.org/wiki/Unicode) uses up to 4 bytes (32 bits) of storage per character, and indexes characters using a base $b =16$ (hexadecimal) numbering system. Hexadecimal numbers are used as a convenience; they are much shorter than their binary equivalents and, thus, are easier for us to read and write than long sequences of 0s and 1s. While Unicode code points are ultimately stored as binary values, hexadecimal provides a direct and convenient way for us (humans) to represent these values.
* Julia has [built-in Unicode support in the standard library](https://docs.julialang.org/en/v1/stdlib/Unicode/#Unicode); thus, we can work with these characters in our programs. For more information on the specific Unicode primitives supported by Julia, check out the [Julia documentation](https://docs.julialang.org/en/v1/manual/unicode-input/).

Let's do some math with emojis (not really that important, but fun):

In [7]:
🌽 = 16; # corn = 16 \:corn: then tab
🍣 = 4; # sushi = 4 \:sushi: then tab

Can we do operations?

In [8]:
🌽 + 🍣  # addition?

20

In [9]:
(🌽 * 🍣) # multiplication?

64

How about logical comparisons? Because Julia has built-in support for [Unicode characters](https://en.wikipedia.org/wiki/Unicode), in addition to using these characters as variables, you can also use many of the Unicode mathematical symbols to _write functional code_ that looks like something we might understand from math class.
* For example, consider `🌽 = 16` and `🍣 = 4`. We know that $🌽\geq{🍣}$ should return `true.` Let's write the `greater than or equal to` check using [Unicode characters](https://en.wikipedia.org/wiki/Unicode):

In [10]:
🌽 ≥ 🍣 # logical comparison? (\geq then tab)

true

We can also determine whether an item is in the given collection. For example, assume we have the character set $\mathbb{C} \equiv \left\{A,B,Q,R,S\right\}$. Let's write an expression to check if $c\in\mathbb{C}$, where $\in$ denotes the `element of` operation, and `c` is some (test) character. 
* Use the [Set data structure](https://docs.julialang.org/en/v1/base/collections/#Base.Set) to do this example. [`Set` is a collection type](https://docs.julialang.org/en/v1/base/collections/#Base.Set) (included in most modern languages) that holds items; sets are `unique` but do not maintain order.

Specify the set $\mathbb{C}$:

In [11]:
C = let
    C = Set{Char}(); # empty set with Char types
    push!(C,'A'); # add a `A` to set C 
    push!(C,'B'); # ... `B` ...
    push!(C,'Q'); # ... `Q` ...
    push!(C,'R'); # ... `R` ...
    push!(C,'S'); # ... `S` ...

    C # return
end

Set{Char} with 5 elements:
  'A'
  'R'
  'S'
  'Q'
  'B'

Specify the test character `c`:

In [12]:
c = 🌽; # Do we have corn in the set ℂ?

Check if $c\in\mathbb{C}$:

In [13]:
c ∈ C # ∈ => \in then tab

false

Ok, so [the Unicode character set is cool](https://en.wikipedia.org/wiki/Unicode), and [Julia's support for Unicode](https://docs.julialang.org/en/v1/manual/unicode-input/) is next level. But how are these characters related to base 16 numbers? Let's dig into this question next.

## Task 3: Unicode Strings and Codepoints
The built-in [Julia `String` type](https://docs.julialang.org/en/v1/base/strings/) is similar (in some ways) to the traditional text model in languages like [C](https://en.wikipedia.org/wiki/C_(programming_language)), namely, the [`String` type](https://docs.julialang.org/en/v1/base/strings/) is an ordered collection (array) of `Char` types. However, the models for the characters are more complicated.

Let's play around with a `test_string_ascii::String`:

In [14]:
test_string_ascii = "Test String in Julia (notice the double quotes). Python uses both single and double quotes for Strings. 😒";

We convert `test_string_ascii::String` to an `Array{Char,1}` collection using [the `collect(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple%7BAny%7D). Each character has a `U+xxxx` type code associated with it. That is a base 16 Unicode code point!
* __What?!?__ In a `U+xxxx` unicode code point, `U+` is followed by a four- or six-digit hexadecimal number `xx..x` uniquely identifying a character in the Unicode standard. However, (really) the `U+xxxx` points are just integers! Thus, we should be able to interconvert between the `U+xxxx` and integer representations.

Let's check that out!

In [15]:
character_array_test_string_ascii = test_string_ascii |> collect

105-element Vector{Char}:
 'T': ASCII/Unicode U+0054 (category Lu: Letter, uppercase)
 'e': ASCII/Unicode U+0065 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 'r': ASCII/Unicode U+0072 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'n': ASCII/Unicode U+006E (category Ll: Letter, lowercase)
 ⋮
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 'r': ASCII/Unicode U+0072 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'n': ASCII/Unicode U+006E (category Ll: Letter, lowercase)
 'g': ASCII/Unicode U+0067 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 '.': ASCII/

### How do we calculate a codepoint?
First, let's try to compute a codepoint (index) for an example Unicode character. Consider `🍣`, which equals `U+1F363`. Let's store a test character in the `test_unicode_char::Char` variable:

In [16]:
test_unicode_char = '🍣' # want to select another Unicode character?

'🍣': Unicode U+1F363 (category So: Symbol, other)

We can compute the `base 10` index for a Unicode character by calling [the `Unicode.julia_chartransform(...)` method](https://docs.julialang.org/en/v1/stdlib/Unicode/#Unicode.julia_chartransform) and then converting that value to an `Int.` What is the base 10 value for the `test_unicode_char::Char`?

In [17]:
test_char_index = Unicode.julia_chartransform(test_unicode_char) |> Int

127843

Couldn't we also just do this:

In [18]:
test_unicode_char |> Int # oh yeah!

127843

So we can get the base 10 representation of the Unicode character. Now let's go from base 10 to the `U+xxxx` representation. Convert the base 10 value to a base 16 (hexadecimal) number and then convert that to the [Unicode index](https://en.wikipedia.org/wiki/Unicode) format. 
> __Hexadecimal digits__: Hexadecimal numbers use decimal digits $(0,1,\dots,9)$ and six extra symbols; the letters `A`, `B`, `C`, `D`, `E`, and `F`, where hexadecimal `A` = decimal 10, through hexadecimal `F` = decimal 15.

Let's start by building the `hexidecimal_digits_dictionary::Dict{Int64, Char}` dictionary which maps the index of a digit to the digit, e.g., `13 => D`.

In [19]:
hexadecimal_digits_dictionary = let

    # initialize 0
    hexadecimal_digits_dictionary = Dict{Int,Char}()
    base = 16; # what base are using?

    # loop: process each digit 0 -> 15
    for i ∈ 0:(base - 1)
        hexadecimal_digits_dictionary[i] = '0' + i |> Char # hmmm! This is wacky; why does this work?
        if (i > 9)
            hexadecimal_digits_dictionary[i] = 'A' + (i - 10) |> Char 
        end
    end
    hexadecimal_digits_dictionary # return
end

Dict{Int64, Char} with 16 entries:
  5  => '5'
  12 => 'C'
  8  => '8'
  1  => '1'
  0  => '0'
  6  => '6'
  11 => 'B'
  9  => '9'
  14 => 'E'
  3  => '3'
  7  => '7'
  4  => '4'
  13 => 'D'
  15 => 'F'
  2  => '2'
  10 => 'A'

#### Algorithm
Next, let's specify and implement our first algorithm to convert a base 10 number into a Unicode character code. [Unicode](https://en.wikipedia.org/wiki/Unicode) left-pads the hexadecimal character value with zeros until a 5-digit code is generated, and then `U+` is appended to the five-digit code.

__Initialize__: Specify a non-negative integer $x\in\mathbb{Z}_{\geq{0}}$ value, and provide the `hexadecimal_digits_dictionary::Dict{Int64, Char}`. Set $i = 0$. Set $q = \infty$.

While $q$ not equal to `0` __do__:
1.  Divide $x$ by `16` and store the quotient in $q$ and the remainder $R\gets{r_{i}}$ in remainder array $R$.
2.  Divide the quotient $q$ by `16` and write down the new quotient $q^{\prime}$. Store the new remainder $R\gets {r_{i}}$ in the remainder array $R$. Set $q\gets{q}^{\prime}$ and update the counter $i\gets {i+1}$.
3.  Repeat step 2 until $q = 0$.

For each element of the remainder array $R$, look up the equivalent hexadecimal digit from the `hexadecimal_digits_dictionary`. Save the hexadecimal digits in the digits array $D$. Starting with the last value of $D$ and moving to the first, write each of the hexadecimal values. Left-pad with `0` characters until the length is `5`.

We've implemented this algorithm below, does it work (do we get back the proper `U+xx..x` code?)

In [20]:
my_code_point = let

    q = test_char_index; 
    remainder_array = Array{Int64,1}();
    while (q != 0)
        r = rem(q,16)
        q = div(q,16)
        push!(remainder_array,r)
    end

    my_code_point = "";
    for i ∈ reverse(remainder_array)
        tmp = hexadecimal_digits_dictionary[i];
        my_code_point *= tmp |> Char;
    end

    # left pad with zeros to get a 4-digit code
    my_code_point = lpad(my_code_point, 5, '0') |> x-> "U+"*x
end

"U+1F363"

## Tests
In the code block below, we check some values in your notebook and give you feedback on which items are correct or different. `Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

> Run the tests below after you've executed the cells that define `test_char_index` and `my_code_point`. They provide immediate pass/fail feedback for basic conversions and will help you confirm that the earlier cells executed correctly.

In [ ]:
let
    @testset verbose = true "CHEME 4/5800 L1d Test Suite" begin

        @testset "Char/Int roundtrip" begin
            @test Int('c') == 99
            @test Char(99) == 'c'
        end

        @testset "Unicode / codepoint checks" begin
            @test isa(test_char_index, Integer)
            @test occursin("U+", my_code_point)
            hexpart = startswith(my_code_point, "U+") ? my_code_point[3:end] : my_code_point
            @test length(hexpart) >= 4
        end
    end
end;

Test Summary:                | Pass  Total  Time
CHEME 4/5800 L1d Test Suite  |    5      5  0.2s
  Char/Int roundtrip         |    2      2  0.2s
  Unicode / codepoint checks |    3      3  0.0s
Total  Time
CHEME 4/5800 L1d Test Suite  |    5      5  0.2s
  Char/Int roundtrip         |    2      2  0.2s
  Unicode / codepoint checks |    3      3  0.0s


Test.DefaultTestSet("CHEME 4/5800 L1d Test Suite", Any[Test.DefaultTestSet("Char/Int roundtrip", Any[], 2, false, false, true, 1.756155203910849e9, 1.756155204079111e9, false, "/Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-1/L1d/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X63sZmlsZQ==.jl"), Test.DefaultTestSet("Unicode / codepoint checks", Any[], 3, false, false, true, 1.756155204079149e9, 1.756155204079217e9, false, "/Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-1/L1d/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X63sZmlsZQ==.jl")], 0, false, true, true, 1.756155203903133e9, 1.756155204079222e9, false, "/Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-1/L1d/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X63sZmlsZQ==.jl")